In [1]:
from arguments import ModelParams, ArgumentParser
from depth_images import calibrate_depth
from gaussian_renderer import GaussianModel
from scene import Scene
import matplotlib.pyplot as plt

parser = ArgumentParser()
dataset = ModelParams(parser)
dataset.model_path = ""
dataset.source_path = "../data/redwood_proc/00002/"
dataset.images = "images"
dataset.resolution = -1
dataset.white_background = False
dataset.eval=True
dataset.num_train_images = 3
gaussians = GaussianModel(dataset.sh_degree)
scene = Scene(dataset, gaussians,shuffle=False)

calibrate_depth(scene)

Reading camera 192/192
Converting point3d.bin to .ply, will happen only the first time you open the scene.
Loading Training Cameras
Loading Test Cameras
Number of points at initialisation :  1810
2.873424081189427 -0.48043364192568055


In [2]:
import open3d as o3d
import numpy as np
from utils.graphics_utils import fov2focal

In [3]:
def camera_to_pcd(camera):
    fx = fov2focal(camera.FoVx,camera.image_width)
    fy = fov2focal(camera.FoVy,camera.image_height)
    intrinics = o3d.camera.PinholeCameraIntrinsic(
        camera.image_width,
        camera.image_height,
        fx,
        fy,
        camera.image_width/2,
        camera.image_height/2,
    )
    print(fx,fy)

    depth_image_o3d = o3d.geometry.Image(camera.depth.numpy()[0].astype(np.float32))
    color_image_o3d = o3d.geometry.Image((camera.original_image.cpu().numpy().transpose((1,2,0))*255).astype(np.uint8))
    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(color_image_o3d,depth_image_o3d,depth_scale=1,depth_trunc=10000,convert_rgb_to_intensity=False)
    
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image,intrinics,camera.world_view_transform.cpu().numpy().T)
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=2, max_nn=30))
    return pcd

all_pcds = []
for i,camera in enumerate(scene.getTrainCameras()):
    pcd = camera_to_pcd(camera)
    all_pcds.append(pcd)

537.2322363094447 535.0870014513293
537.2322363094447 535.0870014513293
537.2322363094447 535.0870014513293


In [4]:
np.asarray(all_pcds[0].points)

array([[-6.54028726,  1.56176424, -2.36879553],
       [-6.55053166,  1.55623133, -2.34527378],
       [-6.56077606,  1.55069843, -2.32175204],
       ...,
       [ 3.82642648,  0.21629888,  4.78538304],
       [ 3.82357113,  0.21475673,  4.7919391 ],
       [ 3.82071577,  0.21321458,  4.79849516]])

In [5]:
def pairwise_registration(source, target, radius):
    print("Apply point-to-plane ICP")
    icp_fine = o3d.registration.registration_colored_icp(
        source, target, radius,
        np.eye(4),
        o3d.registration.ICPConvergenceCriteria(relative_fitness=1e-6,
                                                          relative_rmse=1e-6,
                                                          max_iteration=50))
    
    transformation_icp = icp_fine.transformation
    information_icp = o3d.registration.get_information_matrix_from_point_clouds(
        source, target, radius,
        icp_fine.transformation)
    return transformation_icp, information_icp


def full_registration(pcds, radius):
    pose_graph = o3d.registration.PoseGraph()
    odometry = np.identity(4)
    pose_graph.nodes.append(o3d.registration.PoseGraphNode(odometry))
    n_pcds = len(pcds)
    for source_id in range(n_pcds):
        for target_id in range(source_id + 1, n_pcds):
            transformation_icp, information_icp = pairwise_registration(
                pcds[source_id], pcds[target_id],radius)
            print("Build o3d.registration.PoseGraph")
            if source_id == 0:  # odometry case
                #odometry = np.dot(transformation_icp, odometry)
                pose_graph.nodes.append(
                    o3d.registration.PoseGraphNode(np.linalg.inv(transformation_icp)))
                pose_graph.edges.append(
                    o3d.registration.PoseGraphEdge(source_id,
                                                             target_id,
                                                             transformation_icp,
                                                             information_icp,
                                                             uncertain=False))
            else:  # loop closure case
                pose_graph.edges.append(
                    o3d.registration.PoseGraphEdge(source_id,
                                                             target_id,
                                                             transformation_icp,
                                                             information_icp,
                                                             uncertain=True))
    return pose_graph

def register_pointclouds(all_pcds):
    print("Full registration ...")
    radius = 0.3
    #with o3d.utility.VerbosityContextManager(
    #        o3d.utility.VerbosityLevel.Debug) as cm:
    pose_graph = full_registration(all_pcds,
                                    radius)

    # print("Optimizing PoseGraph ...")
    # option = o3d.registration.GlobalOptimizationOption(
    #     max_correspondence_distance=radius,
    #     edge_prune_threshold=0.25,
    #     reference_node=0)
    # # with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    # o3d.registration.global_optimization(
    #     pose_graph, o3d.registration.GlobalOptimizationLevenbergMarquardt(),
    #     o3d.registration.GlobalOptimizationConvergenceCriteria(), option)

    for point_id in range(len(all_pcds)):
        print(pose_graph.nodes[point_id].pose)
        all_pcds[point_id].transform(pose_graph.nodes[point_id].pose)

In [6]:
def manual_reg(source, target):
  print("Apply point-to-plane ICP")

  threshold = 0.1
  trans_init = np.eye(4)
  
  # result_icp = o3d.registration.registration_icp(
  #     source, target, threshold, trans_init,
  #     o3d.registration.TransformationEstimationPointToPlane())


  result_icp = o3d.registration.registration_colored_icp(
          source, target, threshold, trans_init,
          o3d.registration.ICPConvergenceCriteria(relative_fitness=1e-6,
                                                          relative_rmse=1e-6,
                                                          max_iteration=50))
  print(result_icp)
  print("Transformation is:")
  print(result_icp.transformation)

  trans = np.linalg.inv(result_icp.transformation)
  target.transform(trans)
  return trans

In [12]:
# register_pointclouds(all_pcds)
# save_pcds = all_pcds

import copy

save_pcds = [copy.deepcopy(all_pcds[i]) for i in range(len(all_pcds))]
transformations = [np.eye(4)]
ref_pcd = save_pcds[0]

# for i in range(1,len(all_pcds)):
#     transformations.append(manual_reg(ref_pcd,save_pcds[i]))
#     ref_pcd = ref_pcd+save_pcds[i]

In [13]:
all_points = []
all_colors = []
for i,pcd in enumerate(save_pcds):
    points = np.array(pcd.points)
    colors = np.array(pcd.colors)
    
    # colors = np.zeros_like(points)
    # colors[:,i%3] = 1.0
    # if i>=3:
    #     colors[:,(i+1)%3] = 1.0
    
    all_points.append(points)
    all_colors.append(colors)

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(np.concatenate(all_points))
pcd.colors = o3d.utility.Vector3dVector(np.concatenate(all_colors))
o3d.io.write_point_cloud("test.ply", pcd)

True